# Защита данных клиентов страховой компании

## План выполнения проекта:
### 1. Загрузка и предоработка данных.
### 2. Ответ на вопрос:  Изменится ли качество линейной регрессии, если признаки умножают на обратимую матрицу.
### 3. Алгоритм преобразования и его обоснование.
### 4. Проверка алгоритма.
### 5. Выводы

## 1. Загрузка данных

Загрузим и изучим данные:

In [2]:
import pandas as pd
insurance = pd.read_csv('/datasets/insurance.csv')
#insurance

In [2]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


В таблице 5 столбцов и 5000 строк с данными.

**Признаки** -  пол, возраст и зарплата застрахованного, количество членов его семьи.

**Целевой признак** - количество страховых выплат клиенту за последние 5 лет.

Пропущенных значений нет. Данные в нужном типе, переводить не надо.

Можно проверить на странные значения выбросы, проверить на дубликаты. Названия колонок переделать английскими буквами и перевести в нижний регистр.

## 2. Умножение матриц

#### Ответим на вопрос и обоснуем решение.

Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии? (Её можно обучить заново.)

- a. Изменится. Приведите примеры матриц.
- b. Не изменится. Укажите, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

**Ответ:** Не изменится.

**Обоснование:** 

Предсказания до умножения на обратимую матрицу:        $\mathbf{a = Xw}$

Формула обучения до умножения на обратимую матрицу:       $\mathbf{w = (X^T X)^{-1} X^T y}$

Предсказания после умножения на обратимую матрицу:        $\mathbf{\tilde a = \tilde X \tilde w}$

Формула обучения после умножения на обратимую матрицу:      $\mathbf{\tilde w = (\tilde X^T \tilde X)^{-1} \tilde X^T y}$

Матрица признаков, умноженная на обратимую матрицу: $\mathbf{\tilde X = XP}$

где:

- $X$ и $\tilde X$ — матрица признаков (нулевой столбец состоит из единиц) соответсвенно до умножения на обратимую матрицу и после умножения.

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ и $\tilde w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу) соответсвенно до умножения на обратимую матрицу и после умножения.

Нам нужно проверить выполняется ли равенство: $\mathbf{a = \tilde a}$. 

Для этого запишем формулу в развенутом виде:

$\tilde a = \tilde X \tilde w =  \tilde X (\tilde X^T \tilde X)^{-1} \tilde X^T y $

$a = Xw = X(X^T X)^{-1} X^T y $

Сделаем преобразования:

$\tilde a = \tilde X \tilde w = \tilde X (\tilde X^T \tilde X)^{-1} \tilde X^T y =$

$= XP((XP)^T(XP))^{-1}(XP)^Ty = XP(XP)^{-1}((XP)^T)^{-1}P^TX^Ty =$

$= XPP^{-1}X^{-1}(P^TX^T)^{-1}P^TX^Ty = XEX^{-1}(X^T)^{-1}(P^T)^{-1}P^TX^Ty=$

$ = XX^{-1}(X^T)^{-1}X^Ty = X(X^TX)^{-1}X^Ty = a$

Видим, что при умножении матрицы признаков на обратимую матрицу, предсказания остаются такими же, как и до умножения на обратимую матрицу. Мы доказали, что $\mathbf{a = \tilde a}$.  Значит качество линейной регрессии не должно измениться.

Параметры линейной регрессии в исходной задаче и в преобразованной связаны следующим образом:

$\tilde a = \tilde X \tilde w = XP \tilde w = Xw = a $

$XP \tilde w = Xw$ $\ \ $ $\ \ $ $\Rightarrow $  $\ \ $ $\ \ $ $\tilde w = X^{-1}XP^{-1}w$  $\ \ $ $\ \ $ $\Rightarrow$ $\ \ $ $\ \ $  $\tilde w = P^{-1}w$ 

Вектор весов в преобразованной задаче равен умножению обратной к P матрице на вектор весов в исходной задаче.

## 3. Алгоритм преобразования

**Алгоритм**

Для защиты данных клиентов страховой компании применим умножение матрицы признаков на обратимую матрицу.

**Обоснование**

Нам нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию и при этом при преобразовании качество моделей машинного обучения не ухудшилось.
Из прошлого задания мы поняли, что при умножнии матрицы признаков на обратимую матрицу, качество линейной регрессии не изменяется, это значит, что для преобразования данных подходит такой метод. 

## 4. Проверка алгоритма

Запрограммируем этот алгоритм, применив матричные операции. 

In [3]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [4]:
features = insurance.drop('Страховые выплаты', axis=1)
target = insurance['Страховые выплаты']

In [5]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = ((np.linalg.inv(X.T @ X))@ X.T) @ y
        self.w = w[1:]
        self.w0 = w[0]
    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

##### До преобразования:

In [6]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
r2_score(target, predictions)

0.4249455028666801

##### После преобразования:

Создадим произвольную обратимую матрицу размера  4 на 4:

In [7]:
P = np.random.normal(size=(4,4))

Убедимся в обратимости матрицы:

In [8]:
 P_inv = np.linalg.inv(P) 

При умножениии матрицы на обратную ей должна получиться единичная матрица.

In [9]:
E = P @ P_inv

In [11]:
pd.DataFrame(E)

,0,1,2,3
0,1.000000e+00,-2.567428e-17,2.164030e-17,2.358582e-17
1,5.611908e-16,1.000000e+00,-2.792183e-18,-7.602829e-18
2,-2.437962e-16,5.528889e-17,1.000000e+00,2.328160e-17
3,-1.480655e-15,-2.155467e-16,-1.076970e-16,1.000000e+00


Действительно, получилась единичная матрица, значит сможем дешифровать.

Преобразуем матрицу признаков:

In [12]:
features_changed = features @ P

In [13]:
model = LinearRegression()
model.fit(features_changed, target)
predictions_changed = model.predict(features_changed)
r2_score(target, predictions_changed)

0.42494550131084774

Видим, что качество модели до преобазования и после преобразования совпадает.

Проверим, что качество линейной регрессии из sklearn не отличается до и после преобразования, применим метрику R2.

In [14]:
from sklearn.linear_model import LinearRegression

##### До преобразования:

In [15]:
model = LinearRegression()
model.fit(features, target)
model.predict(features)
r2_score(target, model.predict(features))

0.42494550286668

##### После преобразования:

In [16]:
model = LinearRegression()
model.fit(features_changed, target)
model.predict(features_changed)
r2_score(target, model.predict(features_changed))

0.4249455028666771

Получили тоже самое.

## 5. Выводы:
Задачей данного проекта была защита данных клиентов страховой компании. При преобразовании качество моделей машинного обучения не должно ухудшиться.
Был разработан такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Была обоснована корректность его работы.